In [ ]:
from soyspacing.countbase import CountSpace

model_fname = "./space-correct.model"
model = CountSpace()
model.load_model(model_fname, json_format=False)

In [ ]:
import os
import re
import pandas as pd 
from eunjeon import Mecab

xname = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#7_dropped'}.xlsx")
df = pd.read_excel(xname, index_col=0)

BODY = []
tlist = []

for i in range(len(df)):
    body = df.loc[i, 'body']
    try:
        body = re.sub(r'[\(\[].*?[\)\]]', '', body)
    except:
        BODY.append('')
        continue
    body = re.sub('【.*】','',body)
    body = re.sub('[^0-9가-힣\.\n ]+', '', body)
    body = re.sub('\b[가-하]\.', '', body)
    body = body.split('.')
    body = [e for e in body if len(e.strip()) > 2]
    body = [model.correct(e)[0] for e in body]
    rlist = []
    for e in body:
        result = ''
        for f in e.split():
            if f.isalpha():
                if result == '':
                    result = f
                else:
                    result = result + " "+ f
        rlist.append(re.sub(r'[0-9]', '', result))
    rlist = [e for e in rlist if e]
    tlist = tlist + rlist

    
mecab = Mecab()
text = " ".join(tlist)
text_list = mecab.nouns(text)
word_list = pd.Series(text_list)
words64 = word_list.value_counts().head(64)
words64 = re.findall(r'\w+', words64.to_string())
words64 = [e for e in words64 if e.isalpha()]

In [ ]:
xname = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#7_dropped'}.xlsx")
df = pd.read_excel(xname, index_col=0)

BODY = []

for i in range(10): #len(df)):
    body = df.loc[i, 'body']
    try:
        body = re.sub(r'[\(\[].*?[\)\]]', '', body)
    except:
        BODY.append('')
        continue
    body = re.sub('【.*】','',body)
    body = re.sub('[^0-9가-힣\.\n ]+', '', body)
    body = re.sub('\b[가-하]\.', '', body)
    body = body.split('.')
    body = [e for e in body if len(e.strip()) > 2]
    body = [model.correct(e)[0] for e in body]
    rlist = []
    for e in body:
        result = ''
        for f in e.split():
            if f.isalpha():
                if result == '':
                    if mecab.nouns(f):
                        if mecab.nouns(f)[0] not in words64:
                            result = f
                    else:
                        result = f
                else:
                    if mecab.nouns(f):
                        if mecab.nouns(f)[0] not in words64:
                            result = result + " "+ f
                    else:
                        result = result + " "+ f
                        
        rlist.append(re.sub(r'[0-9]', '', result))
    rlist = [e for e in rlist if e]
    BODY.append(rlist)

df['body'] = BODY
x2name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#8_clean_text'}.xlsx")
df5.to_excel(x2name)